In [1]:
%matplotlib inline
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split,ShuffleSplit
from sklearn.metrics import r2_score
from sklearn import ensemble
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor, FeaturesData, Pool

In [2]:
# ignore warnings from pandas
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_train_raw = pd.read_csv("./data/train.csv")
df_test_raw = pd.read_csv("./data/data_for_test.csv")

In [296]:
#df_train = df_train_raw.groupby(['key']).mean().reset_index()

In [4]:
df_train_raw['xx1_roll'] = df_train_raw['xx1'].rolling(3).median()
df_train_raw['xx2_roll'] = df_train_raw['xx2'].rolling(3).median() 
df_train_raw['xx3_roll'] = df_train_raw['xx3'].rolling(3).median() 
df_train_raw['xx4_roll'] = df_train_raw['xx4'].rolling(3).median() 
df_train_raw['xx5_roll'] = df_train_raw['xx5'].rolling(3).median() 

In [5]:
df_train_raw.head()

,patient_id,key,gender,age,x1,x2,x3,x4,x5,x6,...,xx3,xx4,xx5,y_mean_MAP,y_mean_HR,xx1_roll,xx2_roll,xx3_roll,xx4_roll,xx5_roll
0,1891,1891-3,1,75,22,1,2,0,0,1,...,99.9,49.4,62.5,59.04,80.0,NaN,NaN,NaN,NaN,NaN
1,1891,1891-3,1,75,22,1,2,0,0,1,...,98.4,48.7,61.3,59.04,80.0,NaN,NaN,NaN,NaN,NaN
2,1891,1891-3,1,75,22,1,2,0,0,1,...,95.3,48.5,60.3,59.04,80.0,80.0,95.0,98.4,48.7,61.3
3,1891,1891-3,1,75,22,1,2,0,0,1,...,97.4,48.9,61.3,59.04,80.0,80.0,95.0,97.4,48.7,61.3
4,1891,1891-3,1,75,22,1,2,0,0,1,...,99.6,50.2,62.8,59.04,80.0,80.0,95.0,97.4,48.9,61.3


In [6]:
df_train = df_train_raw.groupby(['key']).mean().reset_index()

In [7]:
df2 = df_train_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[1:2]).reset_index()
df2.columns = ['key','level','xx1_1','xx2_1','xx3_1','xx4_1','xx5_1']
df2.drop(['level'], axis = 1, inplace = True)
df_train = df_train.merge(df2, on = 'key', how = 'left')

df3 = df_train_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[2:3]).reset_index()
df3.columns = ['key','level','xx1_2','xx2_2','xx3_2','xx4_2','xx5_2']
df3.drop(['level'], axis = 1, inplace = True)
df_train = df_train.merge(df3, on = 'key', how = 'left')

df4 = df_train_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[3:4]).reset_index()
df4.columns = ['key','level','xx1_3','xx2_3','xx3_3','xx4_3','xx5_3']
df4.drop(['level'], axis = 1, inplace = True)
df_train = df_train.merge(df4, on = 'key', how = 'left')

df5 = df_train_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[4:5]).reset_index()
df5.columns = ['key','level','xx1_4','xx2_4','xx3_4','xx4_4','xx5_4']
df5.drop(['level'], axis = 1, inplace = True)
df_train = df_train.merge(df5, on = 'key', how = 'left')


In [8]:
df2 = df_train_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[-2:-1]).reset_index()
df2.columns = ['key','level','xx1_2l','xx2_2l','xx3_2l','xx4_2l','xx5_2l']
df2.drop(['level'], axis = 1, inplace = True)
df_train = df_train.merge(df2, on = 'key', how = 'left')

df3 = df_train_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[-3:-2]).reset_index()
df3.columns = ['key','level','xx1_3l','xx2_3l','xx3_3l','xx4_3l','xx5_3l']
df3.drop(['level'], axis = 1, inplace = True)
df_train = df_train.merge(df3, on = 'key', how = 'left')

df4 = df_train_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[-4:-3]).reset_index()
df4.columns = ['key','level','xx1_4l','xx2_4l','xx3_4l','xx4_4l','xx5_4l']
df4.drop(['level'], axis = 1, inplace = True)
df_train = df_train.merge(df4, on = 'key', how = 'left')

df5 = df_train_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[-5:-4]).reset_index()
df5.columns = ['key','level','xx1_5l','xx2_5l','xx3_5l','xx4_5l','xx5_5l']
df5.drop(['level'], axis = 1, inplace = True)
df_train = df_train.merge(df5, on = 'key', how = 'left')

df6 = df_train_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[-6:-5]).reset_index()
df6.columns = ['key','level','xx1_6l','xx2_6l','xx3_6l','xx4_6l','xx5_6l']
df6.drop(['level'], axis = 1, inplace = True)
df_train = df_train.merge(df6, on = 'key', how = 'left')

In [9]:
df1 = df_train_raw.groupby('key').tail(5).reset_index()
ts = df1.groupby(["key"]).agg({'xx1':['median','min','max'],'xx2':['median','min','max'],
                         'xx3':['median','min','max'], 'xx4':['median','min','max'],
                         'xx5':['median','min','max']}).reset_index()
ts.columns = ['key','xx1_10_median','xx1_10_min','xx1_10_max',
             'xx2_10_median','xx2_10_min','xx2_10_max',
             'xx3_10_median','xx3_10_min', 'xx3_10_max',
             'xx4_10_median','xx4_10_min','xx4_10_max',
             'xx5_10_median','xx5_10_min','xx5_10_max']


In [10]:
df_train = df_train.merge(ts, on = 'key', how = 'left')

In [11]:
stats = df_train_raw.groupby(['patient_id','key']).agg({'xx1':['last','first'],'xx2':['last','first'],
                                                        'xx3':['last','first'],'xx4':['last','first'],
                                                        'xx5':['last','first']}).reset_index()
stats.columns = ['patient_id','key','xx1_last','xx1_first','xx2_last','xx2_first',
                 'xx3_last','xx3_first','xx4_last','xx4_first','xx5_last','xx5_first']
stats.drop(['patient_id'], axis = 1, inplace = True)
df_train = df_train.merge(stats, on = 'key', how = 'left')

In [12]:
df_train.head()

,key,patient_id,gender,age,x1,x2,x3,x4,x5,x6,...,xx1_last,xx1_first,xx2_last,xx2_first,xx3_last,xx3_first,xx4_last,xx4_first,xx5_last,xx5_first
0,0-28,0,0,72,39,5,1,0,0,0,...,79.0,82.0,100.0,100.0,112.0,124.0,71.0,69.0,88.0,89.0
1,1-10,1,1,64,55,9,5,1,0,1,...,71.0,64.0,100.0,100.0,109.0,117.0,58.0,61.0,73.0,79.0
2,1-11,1,1,64,55,9,5,1,0,1,...,73.0,70.0,100.0,100.0,117.0,108.0,61.0,58.0,77.0,73.0
3,1-14,1,1,64,55,9,5,1,0,1,...,119.0,87.0,100.0,100.0,116.0,134.0,70.0,68.0,82.0,83.0
4,1-15,1,1,64,55,9,5,1,0,1,...,190.0,123.0,100.0,100.0,133.0,117.0,85.0,71.0,99.0,83.0


In [13]:
df_train.columns

Index(['key', 'patient_id', 'gender', 'age', 'x1', 'x2', 'x3', 'x4', 'x5',
       'x6', 'xx1', 'xx2', 'xx3', 'xx4', 'xx5', 'y_mean_MAP', 'y_mean_HR',
       'xx1_roll', 'xx2_roll', 'xx3_roll', 'xx4_roll', 'xx5_roll', 'xx1_1',
       'xx2_1', 'xx3_1', 'xx4_1', 'xx5_1', 'xx1_2', 'xx2_2', 'xx3_2', 'xx4_2',
       'xx5_2', 'xx1_3', 'xx2_3', 'xx3_3', 'xx4_3', 'xx5_3', 'xx1_4', 'xx2_4',
       'xx3_4', 'xx4_4', 'xx5_4', 'xx1_2l', 'xx2_2l', 'xx3_2l', 'xx4_2l',
       'xx5_2l', 'xx1_3l', 'xx2_3l', 'xx3_3l', 'xx4_3l', 'xx5_3l', 'xx1_4l',
       'xx2_4l', 'xx3_4l', 'xx4_4l', 'xx5_4l', 'xx1_5l', 'xx2_5l', 'xx3_5l',
       'xx4_5l', 'xx5_5l', 'xx1_6l', 'xx2_6l', 'xx3_6l', 'xx4_6l', 'xx5_6l',
       'xx1_10_median', 'xx1_10_min', 'xx1_10_max', 'xx2_10_median',
       'xx2_10_min', 'xx2_10_max', 'xx3_10_median', 'xx3_10_min', 'xx3_10_max',
       'xx4_10_median', 'xx4_10_min', 'xx4_10_max', 'xx5_10_median',
       'xx5_10_min', 'xx5_10_max', 'xx1_last', 'xx1_first', 'xx2_last',
       'xx2_first', '

In [14]:
#drop target column
df_train.drop(['y_mean_HR','y_mean_MAP'], axis = 1, inplace = True)

In [15]:
df_train.shape

(44949, 90)

In [16]:
valid_idx = range(int(df_train.shape[0]*80/100), df_train.shape[0])
train_idx = range(int(df_train.shape[0]*80/100))

In [17]:
#specify Target
y_mean_MAP = df_train_raw[['y_mean_MAP','key']].groupby('key').mean().reset_index().drop(['key'], axis = 1)
y_mean_HR = df_train_raw[['y_mean_HR','key']].groupby('key').mean().reset_index().drop(['key'], axis = 1)

In [18]:
#Train Test Split
X_map_train = df_train.loc[train_idx,:]
X_map_val = df_train.loc[valid_idx,:]
y_map_train = y_mean_MAP.loc[train_idx]
y_map_val = y_mean_MAP.loc[valid_idx]

In [19]:
X_hr_train = df_train.loc[train_idx,:]
X_hr_val = df_train.loc[valid_idx,:]
y_hr_train = y_mean_HR.loc[train_idx]
y_hr_val = y_mean_HR.loc[valid_idx]

In [20]:
#Sanity Check
print(X_map_train.key.isin(X_map_val.key).all())
print(X_hr_train.key.isin(X_hr_val.key).all())

False
False


In [21]:
X_map_train = X_map_train.drop(['key','patient_id'],axis = 1)
X_map_val = X_map_val.drop(['key','patient_id'],axis = 1)
X_hr_train = X_hr_train.drop(['key','patient_id'],axis = 1)
X_hr_val = X_hr_val.drop(['key','patient_id'],axis = 1)

In [22]:
X_map_train.shape

(35959, 88)

In [23]:
X_map_train.head()

,gender,age,x1,x2,x3,x4,x5,x6,xx1,xx2,...,xx1_last,xx1_first,xx2_last,xx2_first,xx3_last,xx3_first,xx4_last,xx4_first,xx5_last,xx5_first
0,0,72,39,5,1,0,0,0,80.166667,99.733333,...,79.0,82.0,100.0,100.0,112.0,124.0,71.0,69.0,88.0,89.0
1,1,64,55,9,5,1,0,1,72.100000,99.800000,...,71.0,64.0,100.0,100.0,109.0,117.0,58.0,61.0,73.0,79.0
2,1,64,55,9,5,1,0,1,71.500000,99.966667,...,73.0,70.0,100.0,100.0,117.0,108.0,61.0,58.0,77.0,73.0
3,1,64,55,9,5,1,0,1,99.133333,100.000000,...,119.0,87.0,100.0,100.0,116.0,134.0,70.0,68.0,82.0,83.0
4,1,64,55,9,5,1,0,1,150.200000,100.000000,...,190.0,123.0,100.0,100.0,133.0,117.0,85.0,71.0,99.0,83.0


### XGBoost on y_mean_MAP

In [71]:
xgb_map = xgb.XGBRegressor(colsample_bytree=0.6,
                 gamma=1.5,                 
                 learning_rate=0.005,
                 max_depth=5,
                 min_child_weight=5,
                 n_estimators=1500,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.7,
                 seed=42)
xgb_map.fit(X_map_train,y_map_train)

[21:23:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=1.5,
             importance_type='gain', learning_rate=0.005, max_delta_step=0,
             max_depth=5, min_child_weight=5, missing=None, n_estimators=1500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.75, reg_lambda=0.45, scale_pos_weight=1, seed=42,
             silent=None, subsample=0.7, verbosity=1)

In [72]:
y_map_val_pred = xgb_map.predict(X_map_val)
r2_score(y_map_val,y_map_val_pred)

0.8917066068871689

### XGBoost on y_mean_HR

In [ ]:
params = {'learning_rate':[0.03,0.01,0.1,0.5],  
          'subsample':[i/10.0 for i in range(6,11)],
            'n_estimator':[200,500,1000,1500], 
            'max_depth': [2,3,4]}

xgb_hr = xgb.XGBRegressor(nthread=-1) 

grid_hr = GridSearchCV(xgb_hr, params)
grid_hr.fit(X_hr_train,y_hr_train)

In [ ]:
print(r2_score(y_hr_val, grid_hr.best_estimator_.predict(X_hr_val)))

In [333]:
xgb_hr = xgb.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.005,
                 max_depth=5,
                 min_child_weight=1.5,
                 n_estimators=2000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)
xgb_hr.fit(X_hr_train,y_hr_train)

[11:00:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4, gamma=0,
             importance_type='gain', learning_rate=0.005, max_delta_step=0,
             max_depth=5, min_child_weight=1.5, missing=None, n_estimators=2000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.75, reg_lambda=0.45, scale_pos_weight=1, seed=42,
             silent=None, subsample=0.6, verbosity=1)

In [334]:
y_hr_val_pred = xgb_hr.predict(X_hr_val)
r2_score(y_hr_val,y_hr_val_pred)

0.9542176689957953

### Test

In [336]:
df_test_raw['xx1_roll'] = df_test_raw['xx1'].rolling(3).mean()
df_test_raw['xx2_roll'] = df_test_raw['xx2'].rolling(3).mean() 
df_test_raw['xx3_roll'] = df_test_raw['xx3'].rolling(3).mean() 
df_test_raw['xx4_roll'] = df_test_raw['xx4'].rolling(3).mean() 
df_test_raw['xx5_roll'] = df_test_raw['xx5'].rolling(3).mean() 

In [337]:
df_test = df_test_raw.groupby(['key']).mean().reset_index()

In [338]:
df_test.head()

,key,patient_id,gender,age,x1,x2,x3,x4,x5,x6,xx1,xx2,xx3,xx4,xx5,xx1_roll,xx2_roll,xx3_roll,xx4_roll,xx5_roll
0,1005-1,1005,0,80,41,4,2,1,0,0,57.866667,98.433333,126.566667,42.800000,71.466667,57.940476,98.416667,127.250000,43.190476,71.988095
1,1005-10,1005,0,80,41,4,2,0,0,0,75.766667,97.333333,126.933333,48.400000,71.866667,75.644444,97.077778,126.166667,48.377778,71.600000
2,1005-11,1005,0,80,41,4,2,1,0,0,65.666667,92.166667,109.000000,48.633333,65.600000,65.866667,92.488889,109.722222,48.655556,65.833333
3,1005-12,1005,0,80,41,4,2,1,0,0,64.600000,94.933333,124.200000,55.400000,75.133333,64.622222,94.688889,124.188889,55.444444,75.055556
4,1005-13,1005,0,80,41,4,2,0,0,0,73.800000,97.400000,134.733333,52.366667,77.200000,73.422222,97.322222,133.777778,52.100000,76.811111


In [339]:
df2 = df_test_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[1:2]).reset_index()
df2.columns = ['key','level','xx1_1','xx2_1','xx3_1','xx4_1','xx5_1']
df2.drop(['level'], axis = 1, inplace = True)
df_test = df_test.merge(df2, on = 'key', how = 'left')

df3 = df_test_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[2:3]).reset_index()
df3.columns = ['key','level','xx1_2','xx2_2','xx3_2','xx4_2','xx5_2']
df3.drop(['level'], axis = 1, inplace = True)
df_test = df_test.merge(df3, on = 'key', how = 'left')

df4 = df_test_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[3:4]).reset_index()
df4.columns = ['key','level','xx1_3','xx2_3','xx3_3','xx4_3','xx5_3']
df4.drop(['level'], axis = 1, inplace = True)
df_test = df_test.merge(df4, on = 'key', how = 'left')

df5 = df_test_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[4:5]).reset_index()
df5.columns = ['key','level','xx1_4','xx2_4','xx3_4','xx4_4','xx5_4']
df5.drop(['level'], axis = 1, inplace = True)
df_test = df_test.merge(df5, on = 'key', how = 'left')


In [340]:
df2 = df_test_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[-2:-1]).reset_index()
df2.columns = ['key','level','xx1_2l','xx2_2l','xx3_2l','xx4_2l','xx5_2l']
df2.drop(['level'], axis = 1, inplace = True)
df_test = df_test.merge(df2, on = 'key', how = 'left')

df3 = df_test_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[-3:-2]).reset_index()
df3.columns = ['key','level','xx1_3l','xx2_3l','xx3_3l','xx4_3l','xx5_3l']
df3.drop(['level'], axis = 1, inplace = True)
df_test = df_test.merge(df3, on = 'key', how = 'left')

df4 = df_test_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[-4:-3]).reset_index()
df4.columns = ['key','level','xx1_4l','xx2_4l','xx3_4l','xx4_4l','xx5_4l']
df4.drop(['level'], axis = 1, inplace = True)
df_test = df_test.merge(df4, on = 'key', how = 'left')

df5 = df_test_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[-5:-4]).reset_index()
df5.columns = ['key','level','xx1_5l','xx2_5l','xx3_5l','xx4_5l','xx5_5l']
df5.drop(['level'], axis = 1, inplace = True)
df_test = df_test.merge(df5, on = 'key', how = 'left')

df6 = df_test_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[-6:-5]).reset_index()
df6.columns = ['key','level','xx1_6l','xx2_6l','xx3_6l','xx4_6l','xx5_6l']
df6.drop(['level'], axis = 1, inplace = True)
df_test = df_test.merge(df6, on = 'key', how = 'left')

In [341]:
df1 = df_test_raw.groupby('key').tail(5).reset_index()
ts = df1.groupby(["key"]).agg({'xx1':['median','min','max'],'xx2':['median','min','max'],
                         'xx3':['median','min','max'], 'xx4':['median','min','max'],
                         'xx5':['median','min','max']}).reset_index()
ts.columns = ['key','xx1_10_median','xx1_10_min','xx1_10_max',
             'xx2_10_median','xx2_10_min','xx2_10_max',
             'xx3_10_median','xx3_10_min', 'xx3_10_max',
             'xx4_10_median','xx4_10_min','xx4_10_max',
             'xx5_10_median','xx5_10_min','xx5_10_max']


In [342]:
df_test = df_test.merge(ts, on = 'key', how = 'left')

In [343]:
stats = df_test_raw.groupby(['patient_id','key']).agg({'xx1':['last','first'],'xx2':['last','first'],
                                                        'xx3':['last','first'],'xx4':['last','first'],
                                                        'xx5':['last','first']}).reset_index()
stats.columns = ['patient_id','key','xx1_last','xx1_first','xx2_last','xx2_first',
                 'xx3_last','xx3_first','xx4_last','xx4_first','xx5_last','xx5_first']
stats.drop(['patient_id'], axis = 1, inplace = True)
df_test = df_test.merge(stats, on = 'key', how = 'left')

In [344]:
df_test.head()

,key,patient_id,gender,age,x1,x2,x3,x4,x5,x6,...,xx1_last,xx1_first,xx2_last,xx2_first,xx3_last,xx3_first,xx4_last,xx4_first,xx5_last,xx5_first
0,1005-1,1005,0,80,41,4,2,1,0,0,...,55.0,59.0,99.0,99.0,116.0,118.0,36.0,38.0,61.0,66.0
1,1005-10,1005,0,80,41,4,2,0,0,0,...,71.0,77.0,100.0,97.0,124.0,130.0,46.0,50.0,68.0,74.0
2,1005-11,1005,0,80,41,4,2,1,0,0,...,65.0,71.0,89.0,94.0,105.0,118.0,46.0,52.0,62.0,71.0
3,1005-12,1005,0,80,41,4,2,1,0,0,...,68.0,67.0,97.0,95.0,108.0,118.0,47.0,72.0,68.0,83.0
4,1005-13,1005,0,80,41,4,2,0,0,0,...,76.0,70.0,99.0,97.0,132.0,132.0,52.0,49.0,76.0,73.0


In [353]:
df_test.columns

Index(['key', 'patient_id', 'gender', 'age', 'x1', 'x2', 'x3', 'x4', 'x5',
       'x6', 'xx1', 'xx2', 'xx3', 'xx4', 'xx5', 'xx1_roll', 'xx2_roll',
       'xx3_roll', 'xx4_roll', 'xx5_roll', 'xx1_1', 'xx2_1', 'xx3_1', 'xx4_1',
       'xx5_1', 'xx1_2', 'xx2_2', 'xx3_2', 'xx4_2', 'xx5_2', 'xx1_3', 'xx2_3',
       'xx3_3', 'xx4_3', 'xx5_3', 'xx1_4', 'xx2_4', 'xx3_4', 'xx4_4', 'xx5_4',
       'xx1_2l', 'xx2_2l', 'xx3_2l', 'xx4_2l', 'xx5_2l', 'xx1_3l', 'xx2_3l',
       'xx3_3l', 'xx4_3l', 'xx5_3l', 'xx1_4l', 'xx2_4l', 'xx3_4l', 'xx4_4l',
       'xx5_4l', 'xx1_5l', 'xx2_5l', 'xx3_5l', 'xx4_5l', 'xx5_5l', 'xx1_6l',
       'xx2_6l', 'xx3_6l', 'xx4_6l', 'xx5_6l', 'xx1_10_median', 'xx1_10_min',
       'xx1_10_max', 'xx2_10_median', 'xx2_10_min', 'xx2_10_max',
       'xx3_10_median', 'xx3_10_min', 'xx3_10_max', 'xx4_10_median',
       'xx4_10_min', 'xx4_10_max', 'xx5_10_median', 'xx5_10_min', 'xx5_10_max',
       'xx1_last', 'xx1_first', 'xx2_last', 'xx2_first', 'xx3_last',
       'xx3_first', 'xx

In [354]:
X_map_train.columns

Index(['gender', 'age', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'xx1', 'xx2',
       'xx3', 'xx4', 'xx5', 'xx1_roll', 'xx2_roll', 'xx3_roll', 'xx4_roll',
       'xx5_roll', 'xx1_1', 'xx2_1', 'xx3_1', 'xx4_1', 'xx5_1', 'xx1_2',
       'xx2_2', 'xx3_2', 'xx4_2', 'xx5_2', 'xx1_3', 'xx2_3', 'xx3_3', 'xx4_3',
       'xx5_3', 'xx1_4', 'xx2_4', 'xx3_4', 'xx4_4', 'xx5_4', 'xx1_2l',
       'xx2_2l', 'xx3_2l', 'xx4_2l', 'xx5_2l', 'xx1_3l', 'xx2_3l', 'xx3_3l',
       'xx4_3l', 'xx5_3l', 'xx1_4l', 'xx2_4l', 'xx3_4l', 'xx4_4l', 'xx5_4l',
       'xx1_5l', 'xx2_5l', 'xx3_5l', 'xx4_5l', 'xx5_5l', 'xx1_6l', 'xx2_6l',
       'xx3_6l', 'xx4_6l', 'xx5_6l', 'xx1_10_median', 'xx1_10_min',
       'xx1_10_max', 'xx2_10_median', 'xx2_10_min', 'xx2_10_max',
       'xx3_10_median', 'xx3_10_min', 'xx3_10_max', 'xx4_10_median',
       'xx4_10_min', 'xx4_10_max', 'xx5_10_median', 'xx5_10_min', 'xx5_10_max',
       'xx1_last', 'xx1_first', 'xx2_last', 'xx2_first', 'xx3_last',
       'xx3_first', 'xx4_last', 'xx4_first',

In [345]:
test_map_pred = xgb_map.predict(df_test.drop(['key','patient_id'],axis = 1))

In [346]:
test_hr_pred = xgb_hr.predict(df_test.drop(['key','patient_id'],axis = 1))
#test_map_pred = model.predict(df_test)

In [347]:
df_test['y_mean_MAP'] = test_map_pred
df_test['y_mean_HR'] = test_hr_pred

In [348]:
df_submit = df_test[['key','y_mean_MAP','y_mean_HR']]

In [349]:
df_submit.head()

,key,y_mean_MAP,y_mean_HR
0,1005-1,63.459949,54.605846
1,1005-10,71.038574,75.385368
2,1005-11,64.454704,65.233551
3,1005-12,70.337440,61.704193
4,1005-13,76.873314,75.233749


In [350]:
df_submit = df_submit.groupby('key').mean()

In [351]:
df_submit.head()

,y_mean_MAP,y_mean_HR
key,,
1005-1,63.459949,54.605846
1005-10,71.038574,75.385368
1005-11,64.454704,65.233551
1005-12,70.337440,61.704193
1005-13,76.873314,75.233749


In [352]:
df_submit.to_csv("no_one_10.csv")